In [1]:
%load_ext autoreload
%autoreload 2

try:
    # install for jupyterlite if needed
    import piplite
    print('installing piplite packages... This may take up to ~5 minutes. If still running, please try refreshing the page.')
    await piplite.install(['mathpad', 'nbformat>=5.6', 'ipywidgets>=7,<8'])
    print('piplite packages installed and ready to go!')
    in_jupyterlite = True

except ImportError:
    # not running in jupyterlite
    in_jupyterlite = False

<div style="display: flex; height: 300px;">

![cart spring pendulum diagram](imgs/double_pendulum.png)

&nbsp;

![cart spring pendulum simulation](imgs/double_pendulum_simulation.gif)
</div>

In [2]:
from mathpad import *

theta = "theta(t)" * rad
phi = "phi(t)" * rad
m = "m" * kg
l = "l" * meters
g = "g" * meters / second ** 2


In [3]:
O = R2("O") * meters

In [4]:
print("Displacement of m1")
r1 = "r_1(t)" @ O
r1_def = r1 == O.from_polar(l, theta - pi/2)
r1_def

Displacement of m1


\vec{r_1}(t) wrt. O = [l*sin(theta(t)), -l*cos(theta(t))] wrt. O <R2 name="O">

In [5]:

print('Velocity of m1')

v1 = diff(r1)
v1

Velocity of m1


Derivative(\vec{r_1}(t), t) wrt. O

In [6]:
print("Displacement of m2")
r2 = "r_2(t)" @ O
r2_def = r2 == r1 + O.from_polar(l, phi - pi/2)
r2_def

Displacement of m2


\vec{r_2}(t) wrt. O = Matrix([
	[ l*sin(phi(t))],
	[-l*cos(phi(t))]]) + \vec{r_1}(t) wrt. O <R2 name="O">

In [7]:
print("Velocity of m2")
v2 = diff(r2)
v2

Velocity of m2


Derivative(\vec{r_2}(t), t) wrt. O

In [8]:
v2[0]

Derivative(\vec{r_2}_{i}(t), t) meters/second

In [9]:
from mathpad.library.mechanic import euler_lagrange, kinetic_energy, gravitational_energy

print("Total Kinetic Energy")
T = kinetic_energy(m=m, v=v1.norm()) + kinetic_energy(m=m, v=v2.norm())
T

Total Kinetic Energy


0.5*m*(Derivative(\vec{r_1}_{i}(t), t)**2 + Derivative(\vec{r_1}_{j}(t), t)**2) + 0.5*m*(Derivative(\vec{r_2}_{i}(t), t)**2 + Derivative(\vec{r_2}_{j}(t), t)**2) joules

In [11]:
print("Total Potential Energy")
V = gravitational_energy(m=m, h=r1.j, g=g) \
    + gravitational_energy(m=m, h=r2.j, g=g)
V

Total Potential Energy


\vec{r_1}_j*g*m + \vec{r_2}_j*g*m kilogram*meters**2/second**2

In [20]:
r1

\vec{r_1}(t) wrt. O

In [18]:
r1.dot(r2)

Dot(Matrix([
[\vec{r_1}_{i}(t)],
[\vec{r_1}_{j}(t)]]), Matrix([
[\vec{r_2}_{i}(t)],
[\vec{r_2}_{j}(t)]])) meters**2

In [16]:
V - T

\vec{r_1}_j*g*m + \vec{r_2}_j*g*m - 0.5*m*(Derivative(\vec{r_1}_{i}(t), t)**2 + Derivative(\vec{r_1}_{j}(t), t)**2) - 0.5*m*(Derivative(\vec{r_2}_{i}(t), t)**2 + Derivative(\vec{r_2}_{j}(t), t)**2) kilogram*meters**2/second**2

In [17]:
print("Theta Dynamics: Lagrange Equation")
theta_dynamics = euler_lagrange(PE=T, KE=V, NCF=0 * N * meter, var=theta)
theta_dynamics

Theta Dynamics: Lagrange Equation


0 = 0 kilogram*meter**2/(radian*second**2)

In [13]:
print("Phi Dynamics: Lagrange Equation")
phi_dynamics = euler_lagrange(PE=T, KE=V, NCF=0 * N * meter, var=phi)
phi_dynamics

Phi Dynamics: Lagrange Equation


0 = 0 kilogram*meter**2/(radian*second**2)

# Simulation Results:

In [14]:
sim_data = simulate_dynamic_system(
    [theta_dynamics, phi_dynamics],
    plot_title="Double Pendulum System Response to Being Dropped From a Height",
    x_final=5, max_step=0.01,
    substitute={
        g: 9.81,
        l: 0.5,
        m: 1
    },
    initial_conditions={
        theta: pi / 3,
        diff(theta): 0,
        phi: 0,
        diff(phi): 0
    },
    record=[theta, phi]
)


Solving subbed Equations...


AssertionError: No Solution Found

In [ ]:
len(sim_data)

3020

In [ ]:
sim_data[0]

(0, [1.0471975511965979, 0.0])

In [ ]:
import numpy as np
from mathpad import viz3d

def coords_to_labelstr(coords: np.ndarray) -> str:
    sign = "+ " if coords[1, 1] >= 0 else ""
    return f"{coords[1, 0]:.2f} {sign}{coords[1, 1]:.2f} j"

O3 = R3("O")
r1_3d = O3[r1[0], r1[1], 0]
r2_3d = O3[r2[0], r2[1], 0]

r1_viz = viz3d.Vec(O3.zeros(), r1_3d, color='b', head_size=0)
r1_text = viz3d.Text(
    lambda s: coords_to_labelstr(s[r1_3d]),
    r1_3d,
    color='b'
)
r2_viz = viz3d.Vec(r1_3d, r2_3d, color='o', head_size=0)
r2_text = viz3d.Text(
    lambda s: coords_to_labelstr(s[r2_3d]),
    r1_3d,
    color='o'
)
pendulum_bobs = viz3d.Point(r1_3d, )

In [ ]:
import k3d
from k3d.factory import plot as k3d_fig
from math import sin, cos
import numpy as np

def theta_phi_to_vec_coords(theta: float, phi: float):
    l = 0.5

    def unit_polar(angle: float):
        return np.array([(0, 0, 0), (sin(angle), -cos(angle), 0)])

    r1 = l * unit_polar(theta)
    r2 = r1[1] + l * unit_polar(phi)
    return r1, r2


r_1, r_2 = theta_phi_to_vec_coords(*sim_data[0][1])
colors = 0x0000ff, 0xFF9900 # blue, orange
origins = [r[0] for r in (r_1, r_2)]
vectors = [r[1] - r[0] for r in (r_1, r_2)]

vecs = k3d.vectors(origins, vectors, colors=[colors[0], colors[0],
                                             colors[1], colors[1]], head_size=0.5)

r_1_text = k3d.text(coords_to_labelstr(r_1), position=r_1[1], label_box=False, color=colors[0], name="r_1 label")
r_2_text = k3d.text(coords_to_labelstr(r_2), position=r_2[1], label_box=False, color=colors[1], name="r_2 label")

m_points = k3d.points([r_1[1], r_2[1]], colors=colors, point_size=0.05)

def title(t: float):
    return f"Double Pendulum System Response to Being Dropped From a Height (t = {t:.2f}s)"

time_text = k3d.text2d(title(t=0), position=(0, 0), label_box=False, color=0x000000, name="time label", is_html=True)

# fig = k3d_fig(camera_no_rotate=True, camera_no_pan=True, camera_no_zoom=True, camera_auto_fit=False)
fig = k3d_fig()
fig += vecs + r_1_text + r_2_text + time_text + m_points
fig

/home/cal/.cache/pypoetry/virtualenvs/mathpad-IMbbmyaw-py3.8/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning:

Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.



Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [ ]:

from time import sleep
sleep(1) # give the js some time to load

# 2d plot - format = [cx, cy, cz, ox, oy, oz, x_up, y_up, z_up]
fig.camera = [0, -0.5, 2, 0, -0.5, 0, 0, 0, -10]
# fig.grid_auto_fit = False
# fig.grid = [-1, -1, 0, 1, 1, 0]
# fig.camera_fov = 100

fig.camera_reset()

In [ ]:
from ipywidgets import FloatSlider

time_scale_slider = FloatSlider(
    value=0.5,
    min=0,
    max=2.0,
    step=0.01,
    description='Time Scale:'
)
time_scale_slider

FloatSlider(value=0.5, description='Time Scale:', max=2.0, step=0.01)

In [ ]:
from time import sleep

# ServerApp.iopub_msg_rate_limit = 10000.0 
time_scale = time_scale_slider.value

t_last = 0
for t, [theta, phi] in sim_data[::100]:
    dt = t - t_last
    t_last = t
    sleep(dt * time_scale)

    time_text.text = title(t)

    r_1, r_2 = theta_phi_to_vec_coords(theta, phi)
 
    vecs.origins = [r[0] for r in (r_1, r_2)]
    vecs.vectors = [r[1] - r[0] for r in (r_1, r_2)]
    m_points.positions = [r_1[1], r_2[1]]

    r_1_text.text = coords_to_labelstr(r_1)
    r_1_text.position = r_1[1]
    r_2_text.text = coords_to_labelstr(r_2)
    r_2_text.position = r_2[1]

    fig.grid = [-1, -1, -1e-10, 1, 1, 1e-10]